# yFinance
Links for References used for this excercise: 
1. https://github.com/ranaroussi/yfinance
2. https://www.geeksforgeeks.org/python/multithreaded-download-of-yahoo-stock-history-with-python-yfinance/
3. for handling dates and time: https://docs.python.org/3/library/datetime.html & https://stackoverflow.com/questions/32490629/getting-todays-date-in-yyyy-mm-dd-in-python

In [ ]:
# yahoo finance data. 
import yfinance as yf
import time
import datetime

# define program start time and dates for use in calculations 
start= time.time()
today = datetime.date.today()
fivedaysago = today - datetime.timedelta(days=15)

# Convert dates to string type data as is required by the yf.download function
strtoday = str(today)
strfivedaysago =str(fivedaysago)

#Checking the formats of the dates are correct
    #print (type(strtoday))
    #print (type(strfivedaysago))
print (f"Today's date is ",today)
print (f"Five days ago was ",fivedaysago)

# define a list of stocks to track
ticker_list =['META','AAPL','AMZN','NFLX','GOOG']

# Define the functon for collecting the data we want.
    # Start date will be 5 days ago as per instructions. 
    # End date will be the date the script is ran on. 
    # Tickers list is the list of stocks to watch. 
    # Threads allows for the multithread download of stock data - this reduces the time taken to run the script.  
    # group_by allows for the data to be organised by different columns. 
data =yf.download( 
    start= strfivedaysago,
    end= strtoday,
    tickers = ticker_list,
    threads=True,
    group_by='ticker',
    auto_adjust = True,
)

# Transpose the data to allow data to be read from dataframe. 
data =data.T 

# I was concerned i was not seeing 5 days worth of data here, 
# But then realised the stock market does not open Saturday or Sunday, so no data exists for those requests and none is returned into the dataframe. 
print(data.shape)

for t in ticker_list: 
    print(t)   
    print(data.loc[t])
    print("\n")

# This segment allows for monitoring the time taken to complete the execution of the program. 
print ('The program takes ', time.time()-start,'seconds.')

Today's date is  2025-10-08
Five days ago was  2025-09-23


[*********************100%***********************]  5 of 5 completed

(25, 11)
META
Date      2025-09-23    2025-09-24    2025-09-25    2025-09-26    2025-09-29  \
Price                                                                          
Open    7.692500e+02  7.575000e+02  7.534500e+02  7.500000e+02  7.487200e+02   
High    7.706000e+02  7.611100e+02  7.567700e+02  7.519300e+02  7.507800e+02   
Low     7.510700e+02  7.525300e+02  7.445500e+02  7.373500e+02  7.391500e+02   
Close   7.554000e+02  7.606600e+02  7.489100e+02  7.437500e+02  7.434000e+02   
Volume  1.087260e+07  8.828200e+06  1.059110e+07  9.696300e+06  9.246800e+06   

Date      2025-09-30    2025-10-01    2025-10-02    2025-10-03    2025-10-06  \
Price                                                                          
Open    7.422500e+02  7.214900e+02  7.225800e+02  7.296300e+02  7.051900e+02   
High    7.429700e+02  7.218500e+02  7.277700e+02  7.310000e+02  7.168800e+02   
Low     7.263000e+02  7.102000e+02  7.181400e+02  7.101800e+02  6.905100e+02   
Close   7.343800e+02  7.1

## End